# 식신 - 양재역으로 검색
- 명칭, 주요메뉴, 평점, 전화번호, 주소

In [1]:
import requests
import pandas as pd
from urllib.parse import quote

- 데이터 가져오기

In [2]:
base_url = 'https://www.siksinhot.com'
url = f'{base_url}/search?keywords={quote("양재역")}'
req = requests.get(url)
html = req.text

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

- 찾으려고 하는 데이터의 태그

In [4]:
lis = soup.select('div.listTy1 > ul > li')
len(lis)

12

In [5]:
li = lis[0]
href = li.select_one('a')['href']
href

'/P/358859'

In [6]:
sub_url = base_url + href
req = requests.get(sub_url)
sub_soup = BeautifulSoup(req.text, 'html.parser')

In [7]:
store = sub_soup.select_one('.title')
info = store.find('h3').get_text()
info

'그릭슈바인 양재역점3.6HOT'

In [8]:
score = store.select_one('h3 > strong').string
score

'3.6'

In [9]:
info.find(score)

10

In [10]:
name = info[:info.find(score)]
name

'그릭슈바인 양재역점'

In [11]:
menu = sub_soup.select('.store_info p')[1].get_text()
menu

'나이트라이프 > 맥주/호프'

In [12]:
tel = sub_soup.select_one('.p_tel p').get_text()
tel

'02-572-6100'

In [13]:
addr = sub_soup.select_one('.txt_adr').string
addr 

'서울특별시 서초구 강남대로 224'

- 한 페이지에 있는 모든 데이터 불러오기

In [49]:
name_list, score_list, menu_list, tel_list, addr_list = [], [], [], [], []
for li in lis:
  href = li.select_one('a')['href']
  sub_url = base_url + href

  req = requests.get(sub_url)
  sub_soup = BeautifulSoup(req.text, 'html.parser')

  try:
    store = sub_soup.select_one('.title')
    info = store.find('h3').get_text()
    score = store.select_one('h3 > strong').string
    name = info[:info.find(score)]
    menu = sub_soup.select('.store_info p')[1].get_text()
    tel = sub_soup.select_one('.p_tel p').get_text()
    addr = sub_soup.select_one('.txt_adr').string

    score_list.append(float(score))
  except:
    score = '평가중'
    name = info[:info.find(score)]
    menu = sub_soup.select('.store_info p')[1].get_text()
    tel = sub_soup.select_one('.p_tel p').get_text()
    addr = sub_soup.select_one('.txt_adr').string
    score_list.append(score)

  name_list.append(name)
  
  menu_list.append(menu)
  tel_list.append(tel)
  addr_list.append(addr)

In [50]:
name_list

['그릭슈바인 양재역점',
 '미스터브리즈 양재역점',
 '동신참치 양재역점',
 '새우닭 양재역점',
 '양재역 옛날짜장',
 '영동족발 3호점',
 '황재벌',
 '김영모과자점 도곡타워점',
 '소호정 본점',
 '산동칼국수',
 '양재정육식당',
 '스타쉐프']

In [51]:
score_list

[3.6, 3.4, 3.0, '평가중', '평가중', 4.1, 3.9, 4.1, 4.0, 3.9, 4.1, 3.9]

In [52]:
menu_list

['나이트라이프 > 맥주/호프',
 '세계음식 > 카페/커피숍',
 '한국음식 > 회',
 '세계음식 > 씨푸드',
 '세계음식 > 중국음식',
 '한국음식 > 족발/보쌈',
 '한국음식 > 장어구이/꼼장어',
 '세계음식 > 베이커리/제과점',
 '한국음식 > 라면/칼국수/국수/수제비',
 '한국음식 > 라면/칼국수/국수/수제비',
 '한국음식 > 꽃등심/등심/육회',
 '세계음식 > 퓨전레스토랑']

In [53]:
tel_list

['02-572-6100',
 '02-576-4333',
 '02-529-7130',
 '02-575-6088',
 '031-503-2550',
 '02-575-0250',
 '02-3471-5454',
 '02-3460-2005',
 '02-579-7282',
 '02-3473-7972',
 '02-577-5804',
 '02-529-8248']

In [54]:
addr_list

['서울특별시 서초구 강남대로 224',
 '서울 서초구 강남대로34길 6',
 '서울 서초구 강남대로 213',
 '서울특별시 서초구 남부순환로356길 52  1층',
 '경기도 시흥시 공단1대로 13 ',
 '서울특별시 서초구 남부순환로 2628-18',
 '서울특별시 서초구 남부순환로347길 42-4 신화빌딩',
 '서울특별시 강남구 언주로30길 10 현대비젼21',
 '서울특별시 서초구 논현로 27',
 '서울특별시 서초구 강남대로37길 63',
 '서울 서초구 바우뫼로35길 3',
 '서울특별시 강남구 논현로38길 38-6']

In [55]:
df = pd.DataFrame({
    '식당명':name_list, '평점':score_list, '메뉴':menu_list,
    '전화번호':tel_list, '주소':addr_list
})
df.set_index('식당명')

,평점,메뉴,전화번호,주소
식당명,,,,
그릭슈바인 양재역점,3.6,나이트라이프 > 맥주/호프,02-572-6100,서울특별시 서초구 강남대로 224
미스터브리즈 양재역점,3.4,세계음식 > 카페/커피숍,02-576-4333,서울 서초구 강남대로34길 6
동신참치 양재역점,3,한국음식 > 회,02-529-7130,서울 서초구 강남대로 213
새우닭 양재역점,평가중,세계음식 > 씨푸드,02-575-6088,서울특별시 서초구 남부순환로356길 52 1층
양재역 옛날짜장,평가중,세계음식 > 중국음식,031-503-2550,경기도 시흥시 공단1대로 13
영동족발 3호점,4.1,한국음식 > 족발/보쌈,02-575-0250,서울특별시 서초구 남부순환로 2628-18
황재벌,3.9,한국음식 > 장어구이/꼼장어,02-3471-5454,서울특별시 서초구 남부순환로347길 42-4 신화빌딩
김영모과자점 도곡타워점,4.1,세계음식 > 베이커리/제과점,02-3460-2005,서울특별시 강남구 언주로30길 10 현대비젼21
소호정 본점,4,한국음식 > 라면/칼국수/국수/수제비,02-579-7282,서울특별시 서초구 논현로 27
